# Evaluate
データセットでエージェントを性能評価。  
定時ジョブなどで実行する事でエージェントの性能を定期的に計測する用途を想定。

# Prepare

In [0]:
import mlflow
import zoneinfo

tz_info = zoneinfo.ZoneInfo("Asia/Tokyo")

# Define Criteria

In [0]:
# テストデータを作成
eval_dataset = [
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "東京都の明日の天気を教えてください。"}
            ]
        },
        "expectations": {"expected_response": "東京都の明日の天気は晴れです"},
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "横浜市の明日の天気を教えてください。"}
            ]
        },
        "expectations": {"expected_response": "横浜市の明日の天気は晴れです"},
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "群馬県の明日の天気を教えてください。"}
            ]
        },
        "expectations": {"expected_response": "群馬県の天気は豪雨です。"},
    },
]

# Package & Evaluate

In [0]:
import os

# パッケージング前に mlflow の依存パッケージ記録機能を有効化
# os.environ["MLFLOW_LOCK_MODEL_DEPENDENCIES"] = "True"

In [0]:
import datetime
import agent
import evaluate

ymd = datetime.datetime.now(tz=tz_info).strftime("%Y%m%d_%H%M%S")
with mlflow.start_run(run_name=f"pkg_{ymd}"):
    # モデルを記録
    res = mlflow.pyfunc.log_model(
        python_model="agent.py", code_paths=["libs/"], name=f"{agent.AGENT_NAME}_{ymd}",
        extra_pip_requirements=[
            "langchain[community]==1.2.3",
            "langchain-google-genai==4.1.3",
            "databricks_langchain==0.12.1",
        ]
    )
    logged_model = mlflow.pyfunc.load_model(res.model_uri)

    # モデルを評価
    mlflow.set_active_model(model_id=res.model_id)
    res = evaluate.eval_responses(logged_model, eval_dataset)
    print(res)